In [1]:
# %conda install -c conda-forge pytorch

In [12]:
import torch

print(f"MPS 장치 지원 여부: {torch.backends.mps.is_built()}")
print(f"MPS 장치 가용 여부: {torch.backends.mps.is_available()}") 

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device: {device}")
device = 'cpu'

MPS 장치 지원 여부: True
MPS 장치 가용 여부: True
device: mps


# Load Data


In [13]:
"""!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip"""

'!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip\n!unzip ml-latest-small.zip'

In [14]:
import pandas as pd 

rating = pd.read_csv("ml-latest-small/ratings.csv")

In [15]:
users = torch.tensor(rating.userId.values)
items = torch.tensor(rating.movieId.values)

# Split Train - Test


In [16]:
from sklearn.model_selection import train_test_split


rating_train, rating_test = train_test_split(rating, test_size=0.2, random_state=42)


# 텐서로 변경 및 GPU 가속 환경 셋팅
users_train = torch.tensor(rating_train.userId.values).to(device)
items_train = torch.tensor(rating_train.movieId.values).to(device)
ratings_train = torch.tensor(rating_train.rating.values).to(torch.float32).to(device)

users_test = torch.tensor(rating_test.userId.values).to(device)
items_test = torch.tensor(rating_test.movieId.values).to(device)
ratings_test = torch.tensor(rating_test.rating.values).to(torch.float32).to(device)

### NCF Model


In [31]:
len(items)

100836

In [27]:
n_users

tensor(611)

In [25]:
n_users = max(users) + 1
n_items = max(items) + 1


In [32]:
n_items = rating.nunique()['movieId'] + 1

In [35]:
import torch.nn as nn

# Regularization 
dropout_rate = 0.3

# 사용할 유저 벡터와 아이템 벡터의 크기
n_factors = 100
 
 # users와 아이템 수
n_users = max(users) + 1
n_items = max(items) + 1 # 이 부분은 고도화 필요 


P = torch.randn(n_users, n_factors, requires_grad=True, device=device)
Q = torch.randn(n_items, n_factors, requires_grad=True, device=device)

# 모델 생성
model = nn.Sequential(
    nn.Linear(n_factors * 2, 64),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32, 8),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(8,1)
).to(device)

# 학습
mse = nn.MSELoss()

optim = torch.optim.Adam(list(model.parameters())+[P, Q], lr = 0.01)

for epoch in range(300):
    model.train()
    x = torch.cat((P[users_train], Q[items_train]), dim = 1)
    h = model(x).squeeze()
    
    cost = mse(h, ratings_train)
    
    optim.zero_grad()
    cost.backward()
    optim.step()
    
    with torch.no_grad():
        model.eval() #  테스트 할 때 드롭아웃 적용 x
        if epoch % 100 == 0:
            x_train = torch.cat((P[users_train], Q[items_train]), dim = 1)
            h_train = model(x_train).squeeze()
            train_mse = mse(h_train, ratings_train)
            
            x_test = torch.cat((P[users_test], Q[items_test]), dim = 1)
            h_test = model(x_test).squeeze()
            test_mse = mse(h_test, ratings_test)
            
            print(f"epoch: {epoch}, train_mse: {train_mse}, test_mse: {test_mse}")

In [37]:
train_mse

tensor(0.6231)

In [36]:
test_mse

tensor(0.8305)